In [3]:
import numpy as np
from numpy import mean, std
import pandas as pd
from sklearn.metrics import accuracy_score, make_scorer, classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score



# Tensorflow
import tensorflow as tf
import tensorflow_datasets as tfds



print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
import time

c:\Users\andly\.conda\envs\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Num GPUs Available:  1


In [4]:
def convert_sample(image):
    #image = tf.image.rgb_to_grayscale(image)
    image = tf.image.resize(image,[32,32]).numpy()
    image = image.reshape(1,-1)
    return image
    24

In [5]:
data_dir = r'C:\Users\andly'
print('Current data dir '+data_dir)

Current data dir C:\Users\andly


In [6]:
tf.random.set_seed(88)
ds1,ds2,ds3 = tfds.load('patch_camelyon',
                    split=['train[:5%]','test[:5%]','validation[:5%]'],
                    data_dir = data_dir,
                    download=False,
                    batch_size=-1, # All data...no batches needed 
                    as_supervised=True, # So that we easily can transform data to numpy format
                    shuffle_files=True)
print('Done Loading Data')



print("Done spliting data")

Done Loading Data
Done spliting data


In [7]:

train_dataset = tfds.as_numpy(ds1) # FULL DATA
train_dataset_image = np.vstack(list(map(convert_sample,train_dataset[0]))) # <-- This is the X
train_dataset_image_Scaled = StandardScaler(with_mean=0, with_std=1).fit_transform(train_dataset_image)
train_dataset_label = train_dataset[1].reshape(-1,) # <-- This is y   
print(f'Shape of training data features (observations,features): {train_dataset_image_Scaled.shape}')
print(f'Shape of training data labels (observations,): {train_dataset_label.shape}')

validation_dataset = tfds.as_numpy(ds3)
validation_dataset_image = np.vstack(list(map(convert_sample,validation_dataset[0])))
validation_dataset_image_Scaled = StandardScaler(with_mean=0, with_std=1).fit_transform(validation_dataset_image)
validation_dataset_label = validation_dataset[1].reshape(-1,) 

test_dataset = tfds.as_numpy(ds2)
test_dataset_image = np.vstack(list(map(convert_sample,test_dataset[0]))) # <-- X_test
test_dataset_image_Scaled = StandardScaler(with_mean=0, with_std=1).fit_transform(test_dataset_image)
test_dataset_label = test_dataset[1].reshape(-1,)
print("Done spliting data")

c:\Users\andly\.conda\envs\venv\lib\site-packages\sklearn\utils\_param_validation.py:541: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(


Shape of training data features (observations,features): (13107, 3072)
Shape of training data labels (observations,): (13107,)


c:\Users\andly\.conda\envs\venv\lib\site-packages\sklearn\utils\_param_validation.py:541: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(


Done spliting data


c:\Users\andly\.conda\envs\venv\lib\site-packages\sklearn\utils\_param_validation.py:541: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(


In [8]:
from xgboost import XGBRegressor, XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.svm import SVC # for Support Vector Classification model
import plotly.express as px  # for data visualization
import plotly.graph_objects as go # for data visualization


# SVM Estimator

# initialize the SVM model
svm = SVC()

# define the parameter grid for grid search
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
# perform grid search using the SVM model and the parameter grid
grid_search = GridSearchCV(svm, param_grid, cv=5, verbose=4)
#grid_search.fit(train_dataset_image_Scaled, train_dataset_label)

# print the best parameters
#print("Best parameters: ", grid_search.best_params_)
  



In [27]:
svm_tuned = SVC(C=0.1, kernel='rbf', random_state=42)

svm_tuned.fit(train_dataset_image_Scaled, train_dataset_label)

# make predictions on the training set
y_train_pred = svm_tuned.predict(train_dataset_image_Scaled)

# Compute the accuracy on the training set
train_accuracy = accuracy_score(train_dataset_label, y_train_pred)
print("Training Accuracy:", train_accuracy)

# Compute the precision on the training set
train_precision = precision_score(train_dataset_label, y_train_pred)
print("Training Precision:", train_precision)

# Compute the recall on the training set
train_recall = recall_score(train_dataset_label, y_train_pred)
print("Training Recall:", train_recall)

# Compute the F1-score on the training set
train_f1 = f1_score(train_dataset_label, y_train_pred)
print("Training F1-score:", train_f1)


# Make predictions on the validation set
y_val_pred = svm_tuned.predict(validation_dataset_image_Scaled)

# Compute the accuracy on the validation set
val_accuracy = accuracy_score(validation_dataset_label, y_val_pred)
print("Validation Accuracy:", val_accuracy)

# Compute the precision on the validation set
val_precision = precision_score(validation_dataset_label, y_val_pred)
print("Validation Precision:", val_precision)

# Compute the recall on the validation set
val_recall = recall_score(validation_dataset_label, y_val_pred)
print("Validation Recall:", val_recall)

# Compute the F1-score on the validation set
val_f1 = f1_score(validation_dataset_label, y_val_pred)
print("Validation F1-score:", val_f1)

# Make predictions on the test set
y_test_pred = svm_tuned.predict(test_dataset_image_Scaled)

# Compute the accuracy on the test set
test_accuracy = accuracy_score(test_dataset_label, y_test_pred)
print("Test Accuracy:", test_accuracy)

# Compute the precision on the test set
test_precision = precision_score(test_dataset_label, y_test_pred)
print("Test Precision:", test_precision)

# Compute the recall on the test set
test_recall = recall_score(test_dataset_label, y_test_pred)
print("Test Recall:", test_recall)

# Compute the F1-score on the test set
test_f1 = f1_score(test_dataset_label, y_test_pred)
print("Test F1-score:", test_f1)

In [11]:
# XGBOOST Estimator
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
#xgb_tuned = XGBClassifier(tree_method="gpu_hist", gpu_id=0, seed=42)

# Define the parameter grid for the grid search
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.2, 0.3],
    'n_estimators': [100, 200, 300],
    
}

# Initialize the XGBClassifier


# Initialize the GridSearchCV
#grid_search = GridSearchCV(estimator=xgb_tuned, param_grid=param_grid, cv=3, verbose=4, return_train_score=True)

# Fit the GridSearchCV to the training data
#grid_search.fit(train_dataset_image_Scaled, train_dataset_label)


xgb_hyper = XGBClassifier(tree_method="gpu_hist", gpu_id=0, learning_rate=0.1, max_depth=5, n_estimators=300)

xgb_hyper.fit(train_dataset_image_Scaled, train_dataset_label)
# Print the best parameters
#print("Best parameters:", grid_search.best_params_)

# Make predictions on the validation set
y_val_pred = xgb_hyper.predict(validation_dataset_image_Scaled)

# Compute the accuracy on the validation set
val_accuracy = accuracy_score(validation_dataset_label, y_val_pred)
print("Validation Accuracy:", val_accuracy)

# Compute the precision on the validation set
val_precision = precision_score(validation_dataset_label, y_val_pred)
print("Validation Precision:", val_precision)

# Compute the recall on the validation set
val_recall = recall_score(validation_dataset_label, y_val_pred)
print("Validation Recall:", val_recall)

# Compute the F1-score on the validation set
val_f1 = f1_score(validation_dataset_label, y_val_pred)
print("Validation F1-score:", val_f1)

# Make predictions on the test set
y_test_pred = xgb_hyper.predict(test_dataset_image_Scaled)

# Compute the accuracy on the test set
test_accuracy = accuracy_score(test_dataset_label, y_test_pred)
print("Test Accuracy:", test_accuracy)

# Compute the precision on the test set
test_precision = precision_score(test_dataset_label, y_test_pred)
print("Test Precision:", test_precision)

# Compute the recall on the test set
test_recall = recall_score(test_dataset_label, y_test_pred)
print("Test Recall:", test_recall)

# Compute the F1-score on the test set
test_f1 = f1_score(test_dataset_label, y_test_pred)
print("Test F1-score:", test_f1)



Validation Accuracy: 0.778998778998779
Validation Precision: 0.7984496124031008
Validation Recall: 0.75
Validation F1-score: 0.7734668335419274
Test Accuracy: 0.7661782661782662
Test Precision: 0.7722513089005235
Test Recall: 0.7384230287859824
Test F1-score: 0.7549584133077414


In [12]:
# Make predictions on the training set
y_train_pred = xgb_hyper.predict(train_dataset_image_Scaled)
# Compute the accuracy on the training set
train_accuracy = accuracy_score(train_dataset_label, y_train_pred)
print("Training Accuracy:", train_accuracy)

# Compute the precision on the training set
train_precision = precision_score(train_dataset_label, y_train_pred)
print("Training Precision:", train_precision)

# Compute the recall on the training set
train_recall = recall_score(train_dataset_label, y_train_pred)
print("Training Recall:", train_recall)

# Compute the F1-score on the training set
train_f1 = f1_score(train_dataset_label, y_train_pred)
print("Training F1-score:", train_f1)

Training Accuracy: 0.9965667200732433
Training Precision: 0.9964858670741024
Training Recall: 0.996638141809291
Training F1-score: 0.9965619986247994


In [161]:
# Ensemble Learning with non-deep learning

import numpy as np
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

# Initialize the classifiers
svm = SVC(kernel='rbf', C=1)
rf = RandomForestClassifier(n_estimators=300)
xgb = XGBClassifier(tree_method="gpu_hist", gpu_id=0, max_depth=5, objective='binary:logistic', n_estimators=300, eta=0.05)
gb = GradientBoostingClassifier(n_estimators=300)

# Create the ensemble model
ensemble = StackingClassifier(estimators=[('svm', svm), ('rf', rf), ('xgb', xgb), ('gb',gb)], final_estimator=LogisticRegression())

# Fit the ensemble model to the training data
ensemble.fit(train_dataset_image_Scaled, train_dataset_label)





0.7515262515262515


In [180]:
# Make predictions on the test data
y_train_pred = ensemble.predict(train_dataset_image_Scaled)
# Compute the accuracy on the training set
train_accuracy = accuracy_score(train_dataset_label, y_train_pred)
print("Training Accuracy:", train_accuracy)

# Compute the precision on the training set
train_precision = precision_score(train_dataset_label, y_train_pred)
print("Training Precision:", train_precision)

# Compute the recall on the training set
train_recall = recall_score(train_dataset_label, y_train_pred)
print("Training Recall:", train_recall)

# Compute the F1-score on the training set
train_f1 = f1_score(train_dataset_label, y_train_pred)
print("Training F1-score:", train_f1)


# Make predictions on the validation set
y_val_pred = ensemble.predict(validation_dataset_image_Scaled)

# Compute the accuracy on the validation set
val_accuracy = accuracy_score(validation_dataset_label, y_val_pred)
print("Validation Accuracy:", val_accuracy)

# Compute the precision on the validation set
val_precision = precision_score(validation_dataset_label, y_val_pred)
print("Validation Precision:", val_precision)

# Compute the recall on the validation set
val_recall = recall_score(validation_dataset_label, y_val_pred)
print("Validation Recall:", val_recall)

# Compute the F1-score on the validation set
val_f1 = f1_score(validation_dataset_label, y_val_pred)
print("Validation F1-score:", val_f1)

# Make predictions on the test set
y_test_pred = ensemble.predict(test_dataset_image_Scaled)

# Compute the accuracy on the test set
test_accuracy = accuracy_score(test_dataset_label, y_test_pred)
print("Test Accuracy:", test_accuracy)

# Compute the precision on the test set
test_precision = precision_score(test_dataset_label, y_test_pred)
print("Test Precision:", test_precision)

# Compute the recall on the test set
test_recall = recall_score(test_dataset_label, y_test_pred)
print("Test Recall:", test_recall)

# Compute the F1-score on the test set
test_f1 = f1_score(test_dataset_label, y_test_pred)
print("Test F1-score:", test_f1)

Training Accuracy: 0.9927519645990692
Training Precision: 0.9871581809941079
Training Recall: 0.9984718826405868
Training F1-score: 0.9927828002734939
Validation Accuracy: 0.753968253968254
Validation Precision: 0.7952314165497896
Validation Recall: 0.6881067961165048
Validation F1-score: 0.7378009108653221
Test Accuracy: 0.7515262515262515
Test Precision: 0.7663043478260869
Test Recall: 0.7058823529411765
Test F1-score: 0.7348534201954398


In [188]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Define the parameter grid for the grid search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# Initialize the RandomForestClassifier
rf = RandomForestClassifier()

# Initialize the GridSearchCV
grid_search2 = GridSearchCV(rf, param_grid, cv=3)

# Fit the GridSearchCV to the training data
grid_search2.fit(train_dataset_image_Scaled, train_dataset_label)

# Print the best parameters
print("Best parameters:", grid_search2.best_params_)



Best parameters: {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 200}


TypeError: BaseSearchCV.predict() takes 2 positional arguments but 3 were given

In [193]:
# New rf with hyperparameters found
rf_hyper = RandomForestClassifier(max_depth=20, min_samples_leaf=2, min_samples_split=10, n_estimators=200)


rf.fit(train_dataset_image_Scaled, train_dataset_label)
# Make predictions on the training set
y_train_pred = rf.predict(train_dataset_image_Scaled)

# Compute the accuracy on the training set
train_accuracy = accuracy_score(train_dataset_label, y_train_pred)
print("Training Accuracy:", train_accuracy)

# Compute the precision on the training set
train_precision = precision_score(train_dataset_label, y_train_pred)
print("Training Precision:", train_precision)

# Compute the recall on the training set
train_recall = recall_score(train_dataset_label, y_train_pred)
print("Training Recall:", train_recall)

# Compute the F1-score on the training set
train_f1 = f1_score(train_dataset_label, y_train_pred)
print("Training F1-score:", train_f1)

# Make predictions on the validation set
y_val_pred = rf.predict(validation_dataset_image_Scaled)

# Compute the accuracy on the validation set
val_accuracy = accuracy_score(validation_dataset_label, y_val_pred)
print("Validation Accuracy:", val_accuracy)

# Compute the precision on the validation set
val_precision = precision_score(validation_dataset_label, y_val_pred)
print("Validation Precision:", val_precision)

# Compute the recall on the validation set
val_recall = recall_score(validation_dataset_label, y_val_pred)
print("Validation Recall:", val_recall)

# Compute the F1-score on the test set
val_f1 = f1_score(validation_dataset_label, y_test_pred)
print("Test F1-score:", val_f1)

# Make predictions on the test set
y_test_pred = rf.predict(test_dataset_image_Scaled)

# Compute the accuracy on the test set
test_accuracy = accuracy_score(test_dataset_label, y_test_pred)
print("Test Accuracy:", test_accuracy)

# Compute the precision on the test set
test_precision = precision_score(test_dataset_label, y_test_pred)
print("Test Precision:", test_precision)

# Compute the recall on the test set
test_recall = recall_score(test_dataset_label, y_test_pred)
print("Test Recall:", test_recall)

# Compute the F1-score on the test set
test_f1 = f1_score(test_dataset_label, y_test_pred)
print("Test F1-score:", test_f1)

Training Accuracy: 1.0
Training Precision: 1.0
Training Recall: 1.0
Training F1-score: 1.0
Validation Accuracy: 0.7435897435897436
Validation Precision: 0.7829131652661064
Validation Recall: 0.6783980582524272
Test F1-score: 0.5009439899307742
Test Accuracy: 0.7356532356532357
Test Precision: 0.754874651810585
Test Recall: 0.6783479349186483
Test F1-score: 0.7145682267633487
